In [19]:
from functions import * 
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.vectorstores import Chroma
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

import os
import tempfile
import uuid
import pandas as pd
import re
from dotenv import load_dotenv

In [20]:
load_dotenv()
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

In [21]:
file_path = "../documents/Laws_of_the Game_2024_25.pdf"
loader = PyPDFLoader(file_path)
pages = loader.load()
document_name = os.path.basename(file_path)
for page in pages:
    page.metadata["document_name"] = document_name
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1500, 
                                               chunk_overlap=200, 
                                               length_function=len, separators = ["\n\n", "\n", ""])
chunks = text_splitter.split_documents(pages)

In [22]:
chunks = calculate_chunk_ids(chunks)

In [23]:
add_to_chroma(chunks, "db_carlos_final", OPENAI_API_KEY)

Number of existing documents in DB: 0
👉 Adding new documents: 287


In [ ]:
def upload_file_in_db(file_path:str, OPENAI_API_KEY:str):
    loader = PyPDFLoader(file_path)
    pages = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1500, 
                                                chunk_overlap=200, 
                                                length_function=len, separators = ["\n\n", "\n", ""])
    chunks = text_splitter.split_documents(pages)   
    chunks = calculate_chunk_ids(chunks)
    add_to_chroma(chunks, "vectorstore_chroma", OPENAI_API_KEY)